In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from scipy import io
import numpy as np
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

import seaborn as sns
sns.set_style("white")

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
train_data_generator = train_datagen.flow_from_directory('dane/data_train',
                                            target_size=(150, 150),
                                            batch_size=32,
                                            class_mode='binary',
                                            classes=['non-trypophobic','trypophobic'])

test_datagen = ImageDataGenerator(rescale=1./255)
test_data_generator = test_datagen.flow_from_directory('dane/data_valid',
                                            target_size=(150, 150),
                                            batch_size=32,
                                            class_mode='binary',
                                            classes=['non-trypophobic','trypophobic'])

Found 1560 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [3]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(128, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:
from keras_sequential_ascii import sequential_model_to_ascii_printout
sequential_model_to_ascii_printout(model)

      OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

          Input   #####   (150, 150, 3)
  Convolution2D    \|/  -------------------       896     0.7%
           relu   #####   (148, 148, 32)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (74, 74, 32)
  Convolution2D    \|/  -------------------     18496    14.2%
           relu   #####   (72, 72, 64)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (36, 36, 64)
  Convolution2D    \|/  -------------------     73856    56.7%
           relu   #####   (34, 34, 128)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (17, 17, 128)
        Flatten   ||||| -------------------         0     0.0%
                  #####   (36992,)
        Dropout    | || -------------------         0     0.0%
                  #####   (36992,)
          Dense   XXXXX -------------------     36993    28.4%
        sigmoid 

In [5]:
from keras_tqdm import TQDMNotebookCallback

In [6]:
history = model.fit_generator(
        train_data_generator,
        samples_per_epoch=2000,
        nb_epoch=5,
        validation_data=test_data_generator,
        nb_val_samples=800)
        #verbose=0,
        #callbacks=[TQDMNotebookCallback(leave_inner=False)])

Epoch 1/5
1976/2000 [============================>.] - ETA: 1s - loss: 0.6556 - acc: 0.6088  

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


2008/2000 [==============================] - 153s - loss: 0.6557 - acc: 0.6076 - val_loss: 0.6508 - val_acc: 0.6500
Epoch 2/5
2008/2000 [==============================] - 115s - loss: 0.6268 - acc: 0.6539 - val_loss: 0.6243 - val_acc: 0.6800

In [ ]:
history3 = model.fit_generator(
        train_data_generator,
        samples_per_epoch=2000,
        nb_epoch=5,
        validation_data=test_data_generator,
        nb_val_samples=800)

Epoch 1/5
1976/2000 [============================>.] - ETA: 1s - loss: 0.5840 - acc: 0.6928  

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


2008/2000 [==============================] - 131s - loss: 0.5822 - acc: 0.6942 - val_loss: 0.6434 - val_acc: 0.6500
Epoch 2/5
2008/2000 [==============================] - 120s - loss: 0.5775 - acc: 0.6982 - val_loss: 0.6066 - val_acc: 0.6775